<a href="https://colab.research.google.com/github/Khaarl/ViZDOOM-PPO/blob/main/ViZDOOM_PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Ensure Google Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

!pip install vizdoom

!pip install gymnasium
!pip install numpy
!pip install opencv-python

# After installation, you might need to restart the runtime for the changes to take effect.
# You can do this by going to Runtime -> Restart runtime in the Colab menu.

# --- Preparation of the Environment (Based on the article's likely next steps) ---

# Import necessary libraries
import gymnasium as gym # Or import gym
import numpy as np
import cv2
from vizdoom import DoomGame, Mode, ScreenFormat, ScreenResolution

# Define a function to create the Doom environment (following the structure likely outlined in the article)
def create_doom_environment(scenario_path, config_path):
    game = DoomGame()
    game.load_config(config_path)
    game.set_doom_scenario_path(scenario_path)
    game.set_doom_map("map01")  # Or whichever map you want

    # Configure screen settings (adjust as needed based on the article)
    game.set_screen_format(ScreenFormat.GRAY8)
    game.set_screen_resolution(ScreenResolution.RES_640X480)

    # Other game settings (as in the article)
    game.set_render_hud(False)
    game.set_render_crosshair(False)
    game.set_render_weapon(True)
    game.set_render_effects_sprites(False)
    game.set_render_decals(False)
    game.set_render_particles(False)
    game.set_window_visible(True) # Set to False for headless training

    game.set_mode(Mode.PLAYER)  # Or Mode.SPECTATOR, Mode.ASYNC_PLAYER, Mode.ASYNC_SPECTATOR

    game.init()

    # Get the available actions
    n_actions = game.get_available_buttons().shape[0]

    return game

# --- Example Usage (Illustrative, adapt based on the article) ---

# Define paths to your scenario and config files.
# Assuming they are in your Google Drive in a folder named 'Doom'
scenario_path = '/content/drive/MyDrive/Doom/basic.wad'  # Replace with your actual path
config_path = '/content/drive/MyDrive/Doom/basic.cfg'   # Replace with your actual path

# Create the Doom environment
game = create_doom_environment(scenario_path, config_path)

# Example of running a few episodes
episodes = 2
for episode in range(episodes):
    game.new_episode()
    while not game.is_episode_finished():
        state = game.get_state()
        # Get the game screen
        screen = state.screen_buffer
        # Perform an action (e.g., a random action)
        n = game.get_available_buttons().shape[0]
        action = np.zeros(n)
        if game.is_episode_finished():
            break
        action[np.random.randint(0, n)] = 1
        game.make_action(action.tolist())
        reward = game.get_last_reward()
        print('reward:', reward)
    print('Episode:', episode, 'finished')

game.close()


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 33.5 MB/s eta 0:00:00


FileDoesNotExistException: File "/content/drive/MyDrive/Doom/basic.cfg | ./scenarios//content/drive/mydrive/doom/basic.cfg | /usr/local/lib/python3.10/dist-packages/vizdoom/scenarios//content/drive/mydrive/doom/basic.cfg" does not exist.